# Обнаружение каверов музыкальных треков

## Задача

Обнаружение треков каверов - важная продуктовая задача, которая может значительно улучшить качество рекомендаций музыкального сервиса и повысить счастье наших пользователей. Если мы умеем с высокой точностью классифицировать каверы и связывать их между собой, то можно предложить пользователю новые возможности для управления потоком треков. Например:

- по желанию пользователя можем полностью исключить каверы из рекомендаций;
- показать все каверы на любимый трек пользователя;
- контролировать долю каверов в ленте пользователя.

## Цель
В этом хакатоне нам предлагается разработать решение, которое:

- может классифицировать треки по признаку кавер-некавер;
- связывать (группировать) каверы и исходный трек;
- находит исходный трек в цепочке каверов.

Нам требуется разработать ML-модель для сопоставления текстов музыкальных произведений и для поиска каверов (вариации обработки оригинала с элементами новой аранжировки) по их текстам. 

Для решения этой задачи мы можем использовать любые открытые источники данных и собирать необходимую вам информацию, соблюдая правила использования сервисов, которые эту информацию предоставляют.

Данные предоставлены заказчиком в трех файлах: covers.json, meta.json, lyrics.json.

### Разметка каверов

Файл covers.json содержит разметку каверов, сделанную редакторами сервиса:

- track_id - уникальный идентификатор трека;
- track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
- original_track_id - уникальный идентификатор исходного трека.

Обратите внимание, что не для всех каверов известны идентификаторы исходных треков!!!

### Метаинформация
Файл meta.json 
- track_id - уникальный идентификатор трека;
- dttm - первая дата появления информации о треке;
- title - название трека;
- language - язык исполнения;
- isrc - международный уникальный идентификатор трека;
- genres - жанры;
- duration - длительность трека;

### Текст песен
Файл lyrics.json
- track_id - уникальный идентификатор трека;
- lyricId - уникальный идентификатор текста;
- text - текст трека.

**Исследование пройдет в 7 этапов:**

* загрузка и первичное ознакомление с данными,
* предварительная обработка,
* полноценный разведочный анализ,
* разработка новых синтетических признаков,
* проверка на мультиколлинеарность,
* отбор финального набора обучающих признаков,
* выбор и обучение моделей,
* итоговая оценка качества предсказания лучшей модели,
* анализ важности ее признаков.
* общий вывод

## Загрузка и первичное ознакомление с данными

Импортируем необходимые для анализа данных библиотеки.

Считаем данные из json-файла в датафрейм и сохраним в переменные.

In [4]:
import json
import os

import warnings
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

RANDOM_STATE = 54321

warnings.filterwarnings("ignore")

### Первичный обзор данных

In [11]:
# разметка каверов
path_covers = 'C:\\Users\\fudou\\Untitled Folder 1\\covers.json'

df_covers = pd.read_json(path_covers, lines=True)
df_covers.sample()

,original_track_id,track_id,track_remake_type
52098,None,bdfa3eed7effceab71091971eb3563e8,COVER


In [12]:
# метаинформация
path_meta = 'C:\\Users\\fudou\\Untitled Folder 1\\/meta.json'

df_meta = pd.read_json(path_meta, lines=True, convert_dates=['dttm'])
df_meta.sample()

,track_id,dttm,title,language,isrc,genres,duration
48478,3cb1f33f3795cdade8b4ed28261b04e5,2012-08-25 18:26:33,You Can Win If You Want,EN,ITAA11000076,[POP],249280.0


In [10]:
# текст трека
path_lyrics = 'C:\\Users\\fudou\\Untitled Folder 1\\/lyrics.json'

df_lyrics = pd.read_json(path_lyrics, lines=True)
df_lyrics.sample()

,lyricId,text,track_id
4399,6d0b984ce3a28daf36a75a00f32098e6,The sun will come out tomorrow\nBet your botto...,84b67225ad48c2fb3c942925c52efabc


In [13]:
# напишем функцию для вывода основной информации о датасетe
def main_info(data):
    print(f'Выведем первые 5 строк датафрейма')
    display(data.head())
    print()

    print(f'Основная информация о датафрейме')
    print()
    display(data.info())
    print()

    print(f'Числовое описание данных')
    display(data.describe().style.background_gradient())
    print()

    print(f'Количество строк-дубликатов')
    display(data.duplicated().sum())
    print()

    print(f'Количество пропусков в процентном соотношении')
    display(pd.DataFrame(round(data.isna().mean()*100,1)).style.background_gradient())
    print()

    print(f'Тип признаков')
    print(data.dtypes)


In [14]:
# посмотрим датасет c каверами
main_info(df_covers)

Выведем первые 5 строк датафрейма


,original_track_id,track_id,track_remake_type
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL
1,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL
2,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL
3,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL
4,None,d6288499d0083cc34e60a077b7c4b3e1,COVER



Основная информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71597 entries, 0 to 71596
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_track_id  4821 non-null   object
 1   track_id           71597 non-null  object
 2   track_remake_type  71597 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


None


Числовое описание данных


...


Количество строк-дубликатов


0


Количество пропусков в процентном соотношении


,0
original_track_id,93.300000
track_id,0.000000
track_remake_type,0.000000



Тип признаков
original_track_id    object
track_id             object
track_remake_type    object
dtype: object


In [18]:
# посмотрим датасет c метаинформацией
main_info(df_meta)

Выведем первые 5 строк датафрейма


,track_id,dttm,title,language,isrc,genres,duration
0,c3b9d6a354ca008aa4518329aaa21380,2021-12-16 21:00:00,Happy New Year,EN,RUB422103970,[DANCE],161120.0
1,c57e3d13bbbf5322584a7e92e6f1f7ff,2021-11-24 14:00:33,Bad Habits,EN,QZN882178276,[ELECTRONICS],362260.0
2,955f2aafe8717908c140bf122ba4172d,2021-11-24 15:37:03,Por Esa Loca Vanidad,None,QZNJZ2122549,"[FOLK, LATINFOLK]",260000.0
3,fae5a077c9956045955dde02143bd8ff,2021-11-24 15:37:03,Mil Lagrimas,None,QZNJZ2166033,"[FOLK, LATINFOLK]",190000.0
4,6bede082154d34fc18d9a6744bc95bf5,2021-11-24 15:37:03,Sexo Humo y Alcohol,None,QZNJZ2122551,"[FOLK, LATINFOLK]",203000.0



Основная информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   track_id  71768 non-null  object        
 1   dttm      71768 non-null  datetime64[ns]
 2   title     71768 non-null  object        
 3   language  21969 non-null  object        
 4   isrc      71455 non-null  object        
 5   genres    71768 non-null  object        
 6   duration  71768 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 3.8+ MB


None


Числовое описание данных


,duration
count,71768.000000
mean,204918.699964
std,85598.543883
min,0.000000
25%,161370.000000
50%,199395.000000
75%,239070.000000
max,5487300.000000



Количество строк-дубликатов


TypeError: unhashable type: 'list'

In [17]:
# посмотрим датасет c текстами
main_info(df_lyrics)

Выведем первые 5 строк датафрейма


,lyricId,text,track_id
0,a951f9504e89759e9d23039b7b17ec14,"Живу сейчас обломами, обломками не той любви\n...",1c4b1230f937e4c548ff732523214dcd
1,0c749bc3f01eb8e6cf986fa14ccfc585,Tell me your fable\nA fable\nTell me your fabl...,0faea89b0d7d6235b5b74def72511bd8
2,e2c8830fbc86e5964478243099eec23a,You're ashamed about all your fears and doubts...,9c6dc41d5ccd9968d07f055da5d8f741
3,e2c8830fbc86e5964478243099eec23a,You're ashamed about all your fears and doubts...,bfd04a73e9cffdf0e282c92219a86ea1
4,7624653ca8522ba93470843c74961b7d,"You showed him all the best of you,\nBut I'm a...",8d70930d09cd239c948408d1317d8659



Основная информация о датафрейме

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11414 entries, 0 to 11413
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   lyricId   11414 non-null  object
 1   text      11414 non-null  object
 2   track_id  11414 non-null  object
dtypes: object(3)
memory usage: 267.6+ KB


None


Числовое описание данных


...


Количество строк-дубликатов


0


Количество пропусков в процентном соотношении


,0
lyricId,0.000000
text,0.000000
track_id,0.000000



Тип признаков
lyricId     object
text        object
track_id    object
dtype: object


**Вывод после первичного обзора данных**

Датасет с разметкой каверов содержит 71597 объектов и 3 столбца, при этом обнаружено 93 % пропусков в столбце с original_track_id - уникальный идентификатор исходного трека. При этом 4468 значения являются уникальными из 4821 имеющихся данных об уникальных id исходных треков. 
Датасет с метаинфромацией содержит 71768 объектов и 7 столбцов, пропуски присутствуют в 2х столбцах:language и  isr (международный id трека).   5
Датасет с текстами песен содержит около 99% уникальных значений текстов и 95% уникальных значений lyricId.Н
названия колонои соответствуют первоначальному описанию дан.  т)Типы данных соответствуют содержанию.   cдaем наборе данных.

In [47]:
# объединим датафреймы по столбцу "track id"
merged_df = pd.merge(df_covers, df_meta, on='track_id',how='left')

# Вывод объединенного датафрейма
merged_df.head()

,original_track_id,track_id,track_remake_type,dttm,title,language,isrc,genres,duration
0,eeb69a3cb92300456b6a5f4162093851,eeb69a3cb92300456b6a5f4162093851,ORIGINAL,2009-11-11 21:00:00,Proud Mary,EN,USFI86900049,"[ROCK, ALLROCK]",187220.0
1,fe7ee8fc1959cc7214fa21c4840dff0a,fe7ee8fc1959cc7214fa21c4840dff0a,ORIGINAL,2009-11-11 21:00:00,Fortunate Son,EN,USFI86900065,"[ROCK, ALLROCK]",137780.0
2,cd89fef7ffdd490db800357f47722b20,cd89fef7ffdd490db800357f47722b20,ORIGINAL,2009-09-21 20:00:00,La Camisa Negra,ES,USUL10400965,"[POP, FOLK, LATINFOLK]",216840.0
3,995665640dc319973d3173a74a03860c,995665640dc319973d3173a74a03860c,ORIGINAL,2009-11-16 21:00:00,The Way I Are,EN,USUM70722806,"[FOREIGNRAP, RAP]",179660.0
4,None,d6288499d0083cc34e60a077b7c4b3e1,COVER,2009-09-17 20:00:00,Extraball,EN,FR8Q10900116,[ELECTRONICS],212620.0


In [49]:
merged_df1 = pd.merge(merged_df, df_lyrics, on='track_id',how='left')

In [52]:
merged_df1.head()
merged_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72571 entries, 0 to 72570
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   original_track_id  5378 non-null   object        
 1   track_id           72571 non-null  object        
 2   track_remake_type  72571 non-null  object        
 3   dttm               72571 non-null  datetime64[ns]
 4   title              72571 non-null  object        
 5   language           22598 non-null  object        
 6   isrc               72242 non-null  object        
 7   genres             72571 non-null  object        
 8   duration           72571 non-null  float64       
 9   lyricId            11097 non-null  object        
 10  text               11097 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 6.6+ MB
